# Gosoft Data Scientist Assesment
## Demand Forecasting

by Tanat Metmaolee

__Goal__: To predict 3 months of item-level sales data at different store locations

## Understand the Objective
1. The Dataset is considered a [Time Series Analysis](https://www.geeksforgeeks.org/time-series-analysis-and-forecasting/)
2. To create/choose an optimal statistics/machine learning model to predict item-level sales in advance.
3. Evaluate and Propose how to deal with future demanding forecast to improve profit margins.

### Time Series Components

1. Trend
2. Seasonality
3. Cyclic
4. Irregular

### Packages

Compiled by Python 3.12.8

In [3]:
# Data Analysis
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Statistics
from scipy import stats
from scipy.stats import zscore
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
from sklearn.metrics import accuracy_score
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import STL

# Time Series Model
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Hyperparameter Tuning
from sklearn.model_selection import GridSearchCV
import optuna

# Machine Learning
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from xgboost.callback import EarlyStopping
import lightgbm as lgb
from sklearn.cluster import KMeans

# etc
from tqdm import tqdm
import joblib
import pickle

/home/bareck/anaconda3/envs/ds_projects/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import warnings

warnings.filterwarnings("ignore")

In [5]:
np.random.seed(66)

## Data Collection

In [6]:
raw_df = pd.read_csv('demand-forecasting/train.csv')

In [7]:
test_df = pd.read_csv('demand-forecasting/test.csv')

## Data Preprocessing/Cleaning

### Dataset Structure

In [8]:
# Preview
raw_df.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [9]:
test_df.head()

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1


In [10]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    913000 non-null  object
 1   store   913000 non-null  int64 
 2   item    913000 non-null  int64 
 3   sales   913000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 27.9+ MB


### Null/Na values check

In [11]:
raw_df.isna().sum()

date     0
store    0
item     0
sales    0
dtype: int64

In [12]:
raw_df.isnull().sum()

date     0
store    0
item     0
sales    0
dtype: int64

### Unique Values

In [13]:
raw_df['item'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

In [14]:
raw_df['store'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

### Dataset Conclusion

`store`, `item` Features are cleaned since they're discrete. 

`sales` Label could have some outliers to be checked.

`date` Feature's format needs to be changed.

### Feature Engineering

#### `date` feature

Use in-built pandas Method -> `pd.to_datetime()` to add new columns as `day` `Month` `Year`

for machine learning models

In [15]:
df = raw_df
df['date'] = pd.to_datetime(df['date'])

We're going to keep the `date` column as well for future analysis

#### Add more features for future use

In [16]:
def create_features(df):
    """
    Create Time-based Features
    """
    
    df = df.copy()
    
    # Datetime Features
    df['day_of_week'] = df['date'].dt.day_of_week  # zero-indexed, start from Monday
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    
    # month & day_of_week cyclical features
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    
    # Lag Features
    for lag_amount in (1, 7, 30, 60, 365):
        df[f'lag_{lag_amount}'] = df.groupby(['store', 'item'])['sales'].shift(lag_amount)
        
    # Rolling Mean Features
    for window in [7, 14, 30, 60]:
        df[f'rolling_mean_{window}'] = df.groupby(['store', 'item'])['sales'].transform(
            lambda x: x.rolling(window=window).mean())
        df[f'rolling_std_{window}'] = df.groupby(['store', 'item'])['sales'].transform(
            lambda x: x.rolling(window=window).std())
        df[f'rolling_max_{window}'] = df.groupby(['store', 'item'])['sales'].transform(
            lambda x: x.rolling(window=window).max())
        df[f'rolling_min_{window}'] = df.groupby(['store', 'item'])['sales'].transform(
            lambda x: x.rolling(window=window).min())
        
    # Target Encoding Features
    # Store average sales
    store_means = df.groupby('store')['sales'].mean().to_dict()
    df['store_mean_sales'] = df['store'].map(store_means)
    
    # Item average sales
    item_means = df.groupby('item')['sales'].mean().to_dict()
    df['item_mean_sales'] = df['item'].map(item_means)
    
    # Store-item average sales
    store_item_means = df.groupby(['store', 'item'])['sales'].mean().to_dict()
    df['store_item_mean_sales'] = df.apply(lambda x: store_item_means.get((x['store'], x['item']), 0), axis=1)
    
    # Month-store and month-item interaction effects
    month_store_means = df.groupby(['month', 'store'])['sales'].mean().to_dict()
    df['month_store_mean_sales'] = df.apply(lambda x: month_store_means.get((x['month'], x['store']), 0), axis=1)
    
    month_item_means = df.groupby(['month', 'item'])['sales'].mean().to_dict()
    df['month_item_mean_sales'] = df.apply(lambda x: month_item_means.get((x['month'], x['item']), 0), axis=1)
    
    return df

In [ ]:
df = create_features(df)

### Data Cube Granularity

#### In-Built pandas Approach

In [ ]:
# Item-Level Drill Down Test
df[(df['store'] == 1) & (df['item'] == 2)].head()

Function Implementation

In [ ]:
def get_df(df, store_number=None, item_number=None):
    """
    """
    if store_number is None and item_number is None:
        return df
    elif item_number is None:
        return df[df['store'] == store_number]
    else:
        return df[(df['store'] == store_number) & (df['item'] == item_number)]

### Anomaly Detection (Outliers)

#### Sales Distribution

Most of the distributions are considered normal

In [ ]:
# store 1 item 1 sample
plt.figure(figsize=(10,8))
sns.displot(get_df(df, 1, 1).sales, color='green')
plt.show()

We might have to detect anomalies on item-level since there is not enough information provided by dataset

Things that might need to be checked:
- Z-Score
- Interquartile

#### Inter Quartile Range

$IQR = Q_3 - Q_1$ 

Outliers Data Point:

$outlier = (Q_1 - 1.5 \times IQR)$ or $(Q_3 + 1.5 \times IQR)$

In [ ]:
plt.figure(figsize=(10, 4))
sns.boxplot(x=get_df(df, 1, 1)['sales'])
plt.show()

In [ ]:
def iqr_outliers(df, column):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    
    outliers = ((df[column] < (q1 - 1.5 * iqr)) | (df[column] > (q3 + 1.5 * iqr)))
    
    return outliers

#### Z-Score

$ z = \frac{(x - \mu)}{\sigma} $

- $z$ = Z-score
- $x$ = the current (sales) value
- $\mu$ = mean
- $\sigma$ = SD

In [ ]:
def z_score_outliers(df, column, threshold=3):
    return np.abs(zscore(df[column])) > threshold

#### Combining Z-Score with IQR

In [ ]:
for (store, item), group in df.groupby(['store', 'item']):
    temp_df_iqr = iqr_outliers(group, 'sales')
    temp_df_zscore = z_score_outliers(group, 'sales')
    df.loc[group.index, 'outlier'] = temp_df_iqr | temp_df_zscore

Percentage of Outliers

In [ ]:
len(df[df['outlier'] == True]) / len(df) * 100

0.49% is consider very low and acceptable.

#### Outliers Handling

Replace `sales` Outliers with __median__ value of each item-level

In [ ]:
for (store, item), group in df.groupby(['store', 'item']):
    median_value = group[group['outlier'] == False]['sales'].median()
    df.loc[group.index[group['outlier']], 'sales'] = int(median_value)  # Cast Median to int type


In [ ]:
plt.figure(figsize=(10, 4))
sns.boxplot(x=get_df(df, 1, 1)['sales'])
plt.show()

## Exploratory Data Analysis (EDA)

1. Overall Data
2. Correlation
3. Seasonality
4. Trend & Decomposition
5. Noise
6. Stationarity

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
store_number = 1    # From 1 to 10
item_number = 1     # From 1 to 50

### Sales Data

### Sample Visualization

Average Sales Sample

In [ ]:
fig = px.line(get_df(df, store_number, item_number), 
              x='date', 
              y='rolling_mean_7',
              title=f'Store {store_number} Item {item_number} Average Sales')
fig.show()

### Correlation

In [ ]:
correlation_matrix = get_df(df, store_number, item_number).drop(['store', 'item', 'outlier'], axis=1).corr()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title(f'Feature Correlation Heatmap of store {store_number} item {item_number}')
plt.show()

#### Autocorrelation (ACF)

In [ ]:
plt.figure(figsize=(10,6))
plot_acf(get_df(df, store_number, item_number)['sales'], lags=70)
plt.ylim(0,1)
plt.xlabel('Lags')
plt.ylabel('Correlation')
plt.title(f'Autocorrelation of Store {store_number} item {item_number}')
plt.show()

All of the items is most likely fall under the same lags which is 7

#### Partial Autocorrelation (PACF)

In [ ]:
plt.figure(figsize=(8,4))
plot_pacf(get_df(df, store_number, item_number)['sales'], lags=50)
plt.ylim(0,1)
plt.xlabel('Lags')
plt.ylabel('Correlation')
plt.title(f'Partial Autocorrelation of Store {store_number} item {item_number}')
plt.show()

### Trend & Seasonality

#### Seasonality

Whole dataset month Seasonality

In [ ]:
px.line(df.set_index("date").resample("ME")["sales"].mean())

Seasonality from Sample (1, 1)

In [ ]:
px.line(get_df(df, 1, 1).iloc[:30].set_index('date')['sales'], title=f"First month's sales store (1, 1)")

In [ ]:
px.line(get_df(df, 1, 1).set_index("date").resample("ME")["sales"].sum(), title='whole dataset sales from item (1, 1)')

#### Trend

In [ ]:
dated_df = df.set_index('date')

In [ ]:
trend_multi_output = seasonal_decompose(get_df(dated_df, 1, 1)['sales'], model='multiplicative').trend
trend_addict_output = seasonal_decompose(get_df(dated_df, 1, 1)['sales'], model='addictive').trend

Both trends are quite Horizontal or slightly Upward

In [ ]:
plt.figure(figsize=(14,4))
plt.grid()
plt.plot(trend_multi_output)
plt.show()

In [ ]:
plt.figure(figsize=(14,4))
plt.grid()
plt.plot(trend_addict_output)
plt.show()

In [ ]:
px.line(df.set_index("date").resample("YE")["sales"].sum())

### Stationarity (for stats model)

#### ADF Test

null-hypothesis of ADF -> the time serie is non-stationary

In [ ]:
def adf_getter(df):
    df = df.dropna()
    
    adf_result = adfuller(df, autolag='AIC')
    print(f'ADF Statistic: {adf_result[0]}')
    print(f'p-value: {adf_result[1]}')
    print(f'n_lags: {adf_result[2]}')
    
    print('Critical Values:')
    
    for key, value in adf_result[4].items():
        print(f'{key}, {value}') 

In [ ]:
def check_stationarity(df):
    df = df.dropna()
    adf_result = adfuller(df)
    p_value = adf_result[1]
    
    return p_value < 0.05

In [ ]:
def adf_p_value(df):
    result = adfuller(df)
    
    return result[1]

In [ ]:
def get_non_stationary_items(df, column='sales'):
    non_station_store_items = []
        
    for (store, item), group in df.groupby(['store', 'item']):
        if not check_stationarity(group[column]):
            non_station_store_items.append((store, item))
    
    return non_station_store_items

In [ ]:
non_station_products = get_non_stationary_items(df)

In [ ]:
print(len(non_station_products))

46 items in the whole-sale (500 items) are non-stationary

#### Differencing

In [ ]:
df['diff_1'] = df.groupby(['store', 'item'])['sales'].transform(lambda x: x.diff())
df['diff_2'] = df.groupby(['store', 'item'])['diff_1'].transform(lambda x: x.diff())

The first non-stationary item is at store: 1, item: 13

In [ ]:
df_1_13 = get_df(df, 1, 13).copy()

In [ ]:
check_stationarity(df_1_13['diff_1'])

#### ACF

In [ ]:
plt.figure(figsize=(10,6))
plot_acf(get_df(df, 1, 13)['diff_1'].dropna(), lags=20)
plt.xlabel('Lags')
plt.ylabel('Correlation')
plt.title(f'Autocorrelation of Store {1} item {13}')
plt.show()

#### PACF

In [ ]:
plt.figure(figsize=(10,6))
plot_pacf(get_df(df, 1, 13)['diff_1'].dropna(), lags=30)
plt.xlabel('Lags')
plt.ylabel('Correlation')
plt.title(f'Partial Autocorrelation of Store {1} item {13}')
plt.show()

#### Box-Cox Transformation

Assumption from Box-Cox Transformation:
- Normal Distribution

In [ ]:
box_cox_df, box_cox_lambda = stats.boxcox(get_df(df, 1, 13)['sales'])

#### Log Transformation

skipped

## Model Building

- Time Series
    1. ARIMA
    2. SARIMAX
    3. Prophet
    5. LSTM
- ML
    1. NNR
    2. Catboost
    3. XGBoost
    4. LightGBM
    5. Random Forest

### Issues
1. Model Selection: to find the quickest and the most accurate.
2. Cross-Validation Method: `Timeseriessplit()` but is this the most optimal?
3. Hyperparameter tuning: for SARIMA, XGBoost, LightGBM
4. Dependent features: for future datapoints, the features (such as `lag`, `rolling_mean`) have to be created dynamically. 

#### Features and Target

In [ ]:
features = ['day_of_week', 'month', 'year', 'lag_1', 'lag_7', 'lag_30', 'rolling_mean_7', 'rolling_mean_30']
target = ['sales']
n_splits = 4

In [ ]:
store_items = df[['store', 'item']].drop_duplicates()

#### Cost Functions

- MAE
- MSE
- RMSE

#### Time Series Cross-Validation

In [ ]:
def time_series_cv(n_splits):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    return tscv

### Stats Model

#### ARIMA

Since our data has seasonality, SARIMA would be more suitable model.

#### SARIMA

In [ ]:
def train_sarima_model(df, item, store, horizon=90):
    subset = df[get_df(df, store, item)]
    subset = subset.set_index("date")["sales"]
    
    tscv = TimeSeriesSplit(n_splits=4)
    
    def objective(trial):
        order = (trial.suggest_int("p", 0, 3),  
                 trial.suggest_int("d", 0, 2),  
                 trial.suggest_int("q", 0, 3))  
        seasonal_order = (trial.suggest_int("P", 0, 3),  
                          trial.suggest_int("D", 0, 2),  
                          trial.suggest_int("Q", 0, 3), 
                          7)  # Weekly seasonality
        errors = []
        
        for train_idx, test_idx in tscv.split(subset):
            train, test = subset.iloc[train_idx], subset.iloc[test_idx]
            model = SARIMAX(train, order=order, seasonal_order=seasonal_order)
            model_fit = model.fit(disp=False)
            forecast = model_fit.forecast(len(test))
            errors.append(np.mean((forecast - test) ** 2))
        
        return np.mean(errors)
    
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)
    best_params = study.best_params
    
    final_model = SARIMAX(subset, 
                           order=(best_params["p"], best_params["d"], best_params["q"]),
                           seasonal_order=(best_params["P"], best_params["D"], best_params["Q"], 7))
    final_fit = final_model.fit(disp=False)
   
    forecast = final_fit.forecast(horizon)
    
    return forecast

As I observed through the experiments, SARIMA requires each product's `p`, `d`, `q` variables (which we need to find them manually or run `auto_arima()` method) that would take too long to execute (or inaccurate with global training). 

Therefore, I decided to use machine learning model instead.

### Machine Learning Models

#### OPTUNA

In [ ]:
def tune_model(trial, model_type, train_X, train_y):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'n_estimators': 500,
        'verbose': -1
    }

    if model_type == "lightgbm":
        params.update({
            'num_leaves': trial.suggest_int('num_leaves', 10, 200),
            'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
            'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        })
        model_class = lgb.LGBMRegressor
    elif model_type == "xgboost":
        params.update({
            'gamma': trial.suggest_loguniform('gamma', 1e-8, 10.0),
            'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
            'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        })
        model_class = xgb.XGBRegressor

    rmse_scores = []
    
    for train_index, test_index in tscv.split(train_X):
        X_train, X_test = train_X.iloc[train_index], train_X.iloc[test_index]
        y_train, y_test = train_y.iloc[train_index], train_y.iloc[test_index]

        model = model_class(**params)
        
        if model_type == "lightgbm":
            model.fit(X_train, y_train, eval_set=[(X_test, y_test)], 
                      callbacks=[lgb.early_stopping(50, verbose=False)])
        elif model_type == "xgboost":
            early_stopping = EarlyStopping(rounds=50, metric_name="rmse", data_name="validation")
            model.fit(
                X_train, y_train, 
                eval_set=[(X_test, y_test)],
                callbacks=[early_stopping]
            )
        
        preds = model.predict(X_test)
        rmse = np.sqrt(root_mean_squared_error(y_test, preds))
        rmse_scores.append(rmse)

    return np.mean(rmse_scores)

#### XGBoost/LightGBM

In [ ]:
def train_models(df, model_type, n_trials=10):
    """ Train and save LightGBM/XGBoost models """
    store_items = df[['store', 'item']].drop_duplicates()
    trained_models = {}

    for store, item in tqdm(store_items.values, desc=f"Training {model_type}"):
        df_subset = get_df(df, store, item).copy()

        if len(df_subset) < 100:
            continue  

        train_X, train_y = df_subset[features], df_subset[target]

        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: tune_model(trial, model_type, train_X, train_y), n_trials=n_trials)

        best_params = study.best_params
        model_class = lgb.LGBMRegressor if model_type == "lightgbm" else xgb.XGBRegressor
        final_model = model_class(**best_params)
        final_model.fit(train_X, train_y)

        if model_type == "lightgbm":
            model_filename = f"lightgbm_training_models/{model_type}_model_store{store}_item{item}.pkl"
            joblib.dump(final_model, model_filename)
        else:
            model_filename = f"xgboost_training_models/{model_type}_model_store{store}_item{item}.pkl"
            joblib.dump(final_model, model_filename)
        trained_models[(store, item)] = final_model

    return trained_models

In [ ]:
tscv = TimeSeriesSplit(n_splits=n_splits)

In [ ]:
# model_lgbm = train_models(df, "lightgbm")

In [ ]:
print(xgb.__version__)

In [ ]:
model_xgb = train_models(df, "xgboost")

In [ ]:
with open("lightgbm_model_store1_item1.pkl", "rb") as f:
    loaded_lgbm_model = pickle.load(f)

FORECAST

In [ ]:
forecast_results = []
for model_type, models in [("lightgbm", models_lgbm), ("xgboost", models_xgb)]:
    for (store, item), model in tqdm(models.items(), desc=f"Forecasting with {model_type}"):
        df_subset = get_df(df, store, item).copy()
        future_dates = pd.date_range(start=df_subset['date'].max() + pd.Timedelta(days=1), periods=90, freq='D')

        future_df = pd.DataFrame({'date': future_dates, 'store': store, 'item': item})
        future_df['day_of_week'] = future_df['date'].dt.dayofweek
        future_df['month'] = future_df['date'].dt.month
        future_df['year'] = future_df['date'].dt.year
        future_df['is_weekend'] = (future_df['day_of_week'] >= 5).astype(int)

        latest_data = df_subset.tail(30)
        for lag in [1, 7, 14, 30]:
            future_df[f'lag_{lag}'] = latest_data['sales'].values[-lag] if len(latest_data) >= lag else 0

        for ma in [7, 30]:
            future_df[f'rolling_mean_{ma}'] = latest_data['sales'].rolling(window=ma).mean().values[-1] if len(latest_data) >= ma else 0

        future_df['sales_predicted'] = model.predict(future_df[features])
        future_df['model'] = model_type
        forecast_results.append(future_df)

In [ ]:
def plot_forecasts(forecast_df):
    """ Compare LightGBM vs XGBoost forecasts """
    plt.figure(figsize=(12, 6))

    sample_store, sample_item = forecast_df.iloc[0]['store'], forecast_df.iloc[0]['item']
    df_sample = forecast_df[(forecast_df['store'] == sample_store) & (forecast_df['item'] == sample_item)]

    sns.lineplot(data=df_sample, x="date", y="sales_predicted", hue="model", palette={"lightgbm": "blue", "xgboost": "red"})

    plt.xlabel("Date")
    plt.ylabel("Predicted Sales")
    plt.title(f"Forecast Comparison for Store {sample_store}, Item {sample_item}")
    plt.xticks(rotation=45)
    plt.legend()
    plt.show()

In [ ]:
forecast(model_lgbm, test_df)

In [ ]:
forecast_results = []
for model_type, models in [("lightgbm", models_lgbm), ("xgboost", models_xgb)]:
    for (store, item), model in tqdm(models.items(), desc=f"Forecasting with {model_type}"):
        df_subset = df[(df['store'] == store) & (df['item'] == item)].copy()
        future_dates = pd.date_range(start=df_subset['date'].max() + pd.Timedelta(days=1), periods=90, freq='D')

        future_df = pd.DataFrame({'date': future_dates, 'store': store, 'item': item})
        future_df['day_of_week'] = future_df['date'].dt.dayofweek
        future_df['month'] = future_df['date'].dt.month
        future_df['year'] = future_df['date'].dt.year
        future_df['is_weekend'] = (future_df['day_of_week'] >= 5).astype(int)

        latest_data = df_subset.tail(30)
        for lag in [1, 7, 14, 30]:
            future_df[f'lag_{lag}'] = latest_data['sales'].values[-lag] if len(latest_data) >= lag else 0

        for ma in [7, 30]:
            future_df[f'rolling_mean_{ma}'] = latest_data['sales'].rolling(window=ma).mean().values[-1] if len(latest_data) >= ma else 0

        future_df['sales_predicted'] = model.predict(future_df[features])
        future_df['model'] = model_type
        forecast_results.append(future_df)

#### Feature Importances

In [ ]:
lgb.plot_importance(lgb_model)

In [ ]:
# Future prediction storage
predictions = []

# Iterate over each store-item combination
for store in df['store'].unique():
    for item in df['item'].unique():
        # Filter historical data for the current store-item combination
        historical_data = df[(df['store'] == store) & (df['item'] == item)].copy()

        # Generate future dates
        future_data = pd.DataFrame({'date': future_dates, 'store': store, 'item': item})
        future_data['day_of_week'] = future_data['date'].dt.dayofweek
        future_data['month'] = future_data['date'].dt.month

        # Initialize placeholders for lag & rolling features
        last_known_sales = historical_data['sales'].tolist()

        future_sales = []
        for i in range(90):
            # Create a single-row DataFrame for prediction
            temp_row = future_data.iloc[i].copy()

            # Compute lag features from last known sales
            temp_row['lag_1'] = last_known_sales[-1] if len(last_known_sales) >= 1 else np.nan
            temp_row['lag_7'] = last_known_sales[-7] if len(last_known_sales) >= 7 else np.nan
            temp_row['lag_30'] = last_known_sales[-30] if len(last_known_sales) >= 30 else np.nan

            # Compute rolling mean features
            temp_row['rolling_mean_7'] = np.mean(last_known_sales[-7:]) if len(last_known_sales) >= 7 else np.nan
            temp_row['rolling_mean_30'] = np.mean(last_known_sales[-30:]) if len(last_known_sales) >= 30 else np.nan

            # Ensure all features exist
            X_future = temp_row[features].to_frame().T  # Convert Series to DataFrame

            X_future['store'] = X_future['store'].astype(int)  # If store numbers are numeric
            X_future['item'] = X_future['item'].astype(int)
            
            lag_features = ['lag_7', 'lag_30', 'rolling_mean_7', 'rolling_mean_30']
            X_future[lag_features] = X_future[lag_features].apply(pd.to_numeric, errors='coerce')
            
            # Predict sales for this day
            predicted_sales = lgb_model.predict(X_future)[0]
            future_sales.append(predicted_sales)

            # Update known sales list for future iterations
            last_known_sales.append(predicted_sales)

        # Store predictions
        predictions.extend(future_sales)

# Store predictions as DataFrame
predictions_df = pd.DataFrame({'date': np.tile(future_dates, len(df['store'].unique()) * len(df['item'].unique())),
                               'store': np.repeat(df['store'].unique(), 90 * len(df['item'].unique())),
                               'item': np.tile(np.repeat(df['item'].unique(), 90), len(df['store'].unique())),
                               'predicted_sales': predictions})

print(predictions_df.head())

## Deployment